# Import

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import os 
import re
import torch
from openpyxl import load_workbook

## STEP 1: Cleaning

In [ ]:
def remove_cross_duplicates(df):
    df['sorted_tuple'] = df.apply(lambda row: tuple(sorted([row['event'], row['knowledge']])), axis=1)

    duplicated_mask = df.duplicated(subset='sorted_tuple', keep='first')
    df_filtered = df[~duplicated_mask].drop(columns=['sorted_tuple'])
    df_removed = df[duplicated_mask].drop(columns=['sorted_tuple'])

    return df_filtered, df_removed

def clean_dataset(file_path, output_path):
    sheets = pd.read_excel(file_path, sheet_name=None)
    cleaned_sheets = {}
    removed_sheets = {}
    
    for country, df in sheets.items():
        df_cleaned = df.drop_duplicates(subset=['event', 'knowledge'])
        df_cleaned = df_cleaned[df_cleaned['event'] != df_cleaned['knowledge']]
        df_cleaned = df_cleaned[~df_cleaned["event"].isin(["oNext", "xNext"]) & 
                                ~df_cleaned["knowledge"].isin(["oNext", "xNext"])]
        
        df_cleaned = df_cleaned.reset_index(drop=True)
        df_result, df_result_removed = remove_cross_duplicates(df_cleaned)
        cleaned_sheets[country] = df_result
        removed_sheets[country] = df_result_removed
    
    with pd.ExcelWriter(output_path) as writer:
        for country, df in cleaned_sheets.items():
            df.to_excel(writer, sheet_name=country, index=False)
    
    print(f"Fichier nettoyé sauvegardé sous : {output_path}")
    return cleaned_sheets, removed_sheets

In [ ]:

input_file = '/Users/junior.tonga/Cultural_Commonsense_Knowledge_Graph/ckg_result_arab/arab_multilingual_extension.xlsx'
output_file = '/Users/junior.tonga/Cultural_Commonsense_Knowledge_Graph/ckg_result_arab/cleaned_arab_multilingual_extension.xlsx'
clean_dataset(input_file, output_file)


## STEP 2: Paths construction 

In [ ]:
def filter_paths_with_relations(paths_with_relations):
    paths_with_relations = sorted(paths_with_relations, key=lambda x: len(x[0]), reverse=True)
    
    unique_paths = []

    for path, relations in paths_with_relations:
        if not any(
            all(node in other_path for node in path) and len(path) < len(other_path)
            for other_path, _ in unique_paths
        ):
            unique_paths.append((path, relations))
    
    return unique_paths

def export_to_excel(file_path, dataframe, sheet_name):
    if os.path.exists(file_path):
        try:
            with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                dataframe.to_excel(writer, sheet_name=sheet_name, index=False)
        except Exception as e:
            print(f"Erreur lors de l'écriture dans le fichier existant : {e}")
    else:
        try:
            with pd.ExcelWriter(file_path, engine='openpyxl',mode='w') as writer:
                dataframe.to_excel(writer, sheet_name=sheet_name, index=False)
        except Exception as e:
            print(f"Erreur lors de la création du fichier Excel : {e}")


In [ ]:
initial_data=pd.read_excel('/Cultural_Commonsense_Knowledge_Graph/results/cleaned_monolingual_generation.xlsx',sheet_name='China')
initial_data

In [ ]:


def compute_statistics_build_paths(file_path, initial_data_path, output_stats_path, output_paths_path, output_format='json',filter=True):
    sheets = pd.read_excel(file_path, sheet_name=None)
    statistics = {}
    
    
    for country, df in (sheets.items()):
        print(country)
        initial_df=pd.read_excel(initial_data_path,sheet_name=country)
        G = nx.DiGraph()
            
        for _, row in df.iterrows():
            G.add_edge(row['event'], row['knowledge'], relation=row['relation'])
            
        sub_topics = df['sub_topic'].unique()
        #topic=df['sub_topic'].unique()
        global_paths_df = pd.DataFrame(columns=['path_with_relations', 'relations', 'sub_topic', 'path_len']) 
        all_nodes = set(df['event']).union(set(df['knowledge'])) 
        country_stats = {
                'num_nodes': len(all_nodes), 
                'num_edges': G.number_of_edges(), 
                'num_assertions': G.number_of_edges(),
                'subtopics': {}
            }
            
        paths_data = []
            
        for sub_topic in sub_topics:
            print(sub_topic)
            df_sub = df[df['sub_topic'] == sub_topic]
            df2 = initial_df[initial_df['sub_topic'] == sub_topic]
            G_sub = nx.DiGraph()
            for _, row in df_sub.iterrows():
                G_sub.add_edge(row['event'], row['knowledge'], relation=row['relation'])
                
            remaining_nodes = set(G_sub.nodes)
            missing_after_cleaning = set(df2['event'].unique()) - remaining_nodes
            if missing_after_cleaning:
                print(f"❌ Missing nodes after transformation: {missing_after_cleaning}")
            else:
                print("✅ All source nodes are present.")
            num_nodes = G_sub.number_of_nodes()
            num_edges = G_sub.number_of_edges()
            degree_distribution = dict(G_sub.degree())
            average_degree = sum(degree_distribution.values()) / num_nodes if num_nodes > 0 else 0
            central_nodes = sorted(degree_distribution.items(), key=lambda x: x[1], reverse=True)[:5]
                
            paths_with_relations = []
            for source in df2['event'].unique():
                for target in G_sub.nodes:
                    print(target)
                    if source != target:
                        try:
                            paths = list(nx.all_simple_paths(G_sub, source=source, target=target))
                            for path in paths:
                                #len(path)=number node in the path
                                if len(path) >= 2:
                                    relations = [G_sub.get_edge_data(path[i], path[i+1])['relation'] for i in range(len(path) - 1)]
                                    paths_with_relations.append((path, relations))
                        except nx.NodeNotFound:
                            continue
                
            if filter==True:
                paths_with_relations_filtered = filter_paths_with_relations(paths_with_relations)
            else:
                paths_with_relations_filtered = paths_with_relations
            paths_data.extend([
                    {
                        'path_with_relations': '\n'.join([f"{path[i]}---{relations[i]}--->{path[i+1]}" for i in range(len(path)-1)]),
                        'sub_topic': sub_topic,
                        'relations': " - ".join(str(r) for r in relations ),
                        'path_len': len(path) - 1 
                    }
                    for path, relations in paths_with_relations_filtered
                ])
                
            country_stats['subtopics'][sub_topic] = {
                    'num_nodes': num_nodes,
                    'num_edges': num_edges,
                    'average_degree': average_degree,
                    'central_nodes': central_nodes,
                    'num_paths': len(paths_with_relations_filtered)
                }
            
        statistics[country] = country_stats
            
            
        if paths_data:
            paths_df = pd.DataFrame(paths_data)
            paths_df = paths_df.dropna() 
            global_paths_df = pd.concat([global_paths_df, paths_df], ignore_index=True)
            export_to_excel(file_path=output_paths_path,dataframe=global_paths_df,sheet_name=country)

    #Save some statistics
    if output_format == 'json':
        with open(output_stats_path, 'w', encoding='utf-8') as f:
            json.dump(statistics, f, indent=4, ensure_ascii=False) 

In [ ]:

input_file = '/Cultural_Commonsense_Knowledge_Graph/ckg_result_arab/cleaned_arab_multilingual_extension.xlsx'
output_stats_file = '/Cultural_Commonsense_Knowledge_Graph/ckg_result_arab/cleaned_arab_multilingual_extension_statistics.json'
output_paths_file = '/Cultural_Commonsense_Knowledge_Graph/ckg_result_arab/ckg_data/cleaned_arab_multilingual_extension_paths_final.xlsx'
initial_data_path='/Cultural_Commonsense_Knowledge_Graph/ckg_result_arab/cleaned_arab_initial_multilingual_generation.xlsx'

In [ ]:
compute_statistics_build_paths(file_path=input_file,initial_data_path=initial_data_path, 
                   output_stats_path=output_stats_file, output_paths_path=output_paths_file, output_format='json',filter=True)

## [OPTIONAL] Reformatted the paths by removing node relationships and extracting only the first node.

In [ ]:
def format_path(path):
    if not path or pd.isna(path):
        return None
    
    steps = path.split(" --> ")
    formatted_steps = [f"- {steps[i]} → {steps[i+1]}" for i in range(len(steps) - 1)]
    
    return "\n".join(formatted_steps)

def extract_first_node_and_reformat(path):
    """
    Extracts the first node and reformats the path by removing relations, avoiding node repetition.

    Args:
    - path (str): A full path string with relations and line breaks.

    Returns:
    - tuple: (first_node, cleaned_path)
    """
    if pd.isna(path) or not isinstance(path, str) or path.strip() == "":
        return "", ""

    # Split by newline first to separate different transitions
    lines = path.split("\n")

    # Extract first node (from the first assertion before "---")
    first_node = lines[0].split("---")[0].strip()

    # Process each transition and clean up relations
    clean_nodes = []
    for line in lines:
        # Remove relations like ---xNext--->
        parts = re.split(r"---[a-zA-Z]+--->", line)
        if len(parts) == 2:  # Ensures it's a valid transition
            node = parts[1].strip()  # Keep only the second part (destination)
            clean_nodes.append(node)

    # Remove duplicate nodes while preserving order
    seen = set()
    unique_nodes = [first_node]  # Start with the first node
    for node in clean_nodes:
        if node not in seen:
            unique_nodes.append(node)
            seen.add(node)

    # Join nodes into a single path
    reformatted_path = " --> ".join(unique_nodes)
    reformatted_path=format_path(reformatted_path)

    return first_node, reformatted_path

In [ ]:

# File paths
input_file = "/Cultural_Commonsense_Knowledge_Graph/ckg_result_arab/ckg_data/cleaned_arab_multilingual_extension_paths_final.xlsx"
output_file = "/Cultural_Commonsense_Knowledge_Graph/ckg_result_arab/ckg_data/cleaned_arab_multilingual_extension_paths_final_reformated.xlsx"

# Read the Excel file with multiple sheets
sheets_dict = pd.read_excel(input_file, sheet_name=None)

# Process each sheet
for sheet_name, df in sheets_dict.items():
    if "path_with_relations" in df.columns:
        # Apply the function and store results in new columns
        df[['first_node', 'paths_without_relation']] = df['path_with_relations'].apply(
            lambda x: pd.Series(extract_first_node_and_reformat(x)) if pd.notna(x) else pd.Series(["", ""])
        )
        sheets_dict[sheet_name] = df

# Save the cleaned data back to Excel
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name, df in sheets_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f":white_check_mark: The reformatted file has been saved at: {output_file}")